In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#라이브러리 설치
pip install split-folders

In [ ]:
#Import the libraries
import zipfile
import os
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/archive (2)_.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp/dataset') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

In [ ]:
data_dir = '../tmp/dataset/PokemonData'

classes = os.listdir(data_dir)
print(os.listdir(data_dir))
print(len(classes))

In [ ]:
import random
import numpy as np
import os
import cv2
import glob
from PIL import Image
import PIL.ImageOps

#다음 변수를 수정하여 새로 만들 이미지 갯수를 정합니다.
num_augmented_images = 50

file_path = '/tmp/dataset/PokemonData/'
dir_names = os.listdir(file_path)
total_origin_dir_num = len(dir_names)
print(len(dir_names))
augment_cnt = 1


for i in range(0, total_origin_dir_num):
    dir_name = dir_names[i]
    file_names=os.listdir(file_path+dir_name)
    total_origin_image_num=len(file_names)          #폴더당 이미지 개수
    for k in range(0, total_origin_image_num):      #폴더의 이미지만큼 반복
      change_picture_index = k
      file_name = file_names[change_picture_index]  #전환할 이미지
      origin_image_path = '/tmp/dataset/PokemonData/'+dir_name +'/'+file_name
      print(origin_image_path)
      image = Image.open(origin_image_path)
      random_augment = random.randrange(1,3)
      save_image_path='/tmp/dataset/PokemonData/'+dir_name+'/'
      if(random_augment == 1):
          #이미지 좌우 반전
          print("invert")
          inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
          inverted_image.save(save_image_path + 'inverted_' + str(augment_cnt) + '.png')   
      elif(random_augment == 2):
          #이미지 기울이기
          print("rotate")
          rotated_image = image.rotate(random.randrange(-20, 20))
          rotated_image.save(save_image_path + 'rotated_' + str(augment_cnt) + '.png')

      augment_cnt += 1  #파일 개수
    augment_cnt=1#파일 개수 초기화

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import splitfolders

In [ ]:
#저장 경로를 drive로 해서 최초 1번만 실행
splitfolders.ratio("/tmp/dataset/PokemonData", output="/content/drive/MyDrive/preprocessing",seed=1337, ratio=(.8, .18, .02), group_prefix=None, move=False)

In [ ]:
tf= transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))
])

#미리 드라이브에 저장해놓은 데이터를 로드
traindata=datasets.ImageFolder(root='/content/drive/MyDrive/preprocessing/train',transform=tf)
testdata=datasets.ImageFolder(root='/content/drive/MyDrive/preprocessing/val',transform=tf)

In [ ]:
batch_size=32
trainloader=DataLoader(traindata,batch_size=batch_size, shuffle=True)
testloader=DataLoader(testdata,batch_size=batch_size, shuffle=True)
train_images,train_labels=iter(trainloader).next()
print(train_labels)

In [ ]:
import numpy as np
import pandas as pd
import os
import torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = '/content/drive/MyDrive/ColabNotebooks/PokemonData'




Mounted at /content/drive


#ResNet50

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch

resnet50 = models.resnet50(pretrained = True)

for param in resnet50.parameters():   #가중치 freeze
  param.requires_grad = False

num_classes = 150
num_ftrs = resnet50.fc.in_features

resnet50.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet50.to(device)

criterion = nn.CrossEntropyLoss() 

optimizer_ft = optim.SGD(resnet50.parameters(), lr=0.001,momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size =7, gamma=0.1)

In [ ]:
from tqdm import tqdm

def train(dataloader, model, loss_fn, optimizer):
    model.train()
    pbar = tqdm(enumerate(dataloader), total = len(dataloader)) #
    learning_loss = 0
    sum_loss = 0

    for batch, (X, y) in pbar:
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss = sum_loss + loss.item() #
        learning_loss = sum_loss / (batch + 1)
        
        description = f'train loss : {round(learning_loss, 4)}'
        pbar.set_description(description)

def test(dataloader, model, loss_fn):
    model.eval()  #
    pbar = tqdm(enumerate(dataloader), total = len(dataloader)) #
    learning_loss = 0
    sum_loss = 0
    learning_acc = 0
    sum_acc = 0
    correct = 0

    for batch, (X, y) in pbar:
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = loss_fn(pred, y)
      correct = (pred.argmax(1)==y).type(torch.float).sum().item()
      acc = correct / batch_size
      sum_acc += acc
      learning_acc = sum_acc / (batch + 1)
      sum_loss = sum_loss + loss.item() #
      learning_loss = sum_loss / (batch + 1)

      description = f'test loss : {learning_loss}  || test acc : {learning_acc}'
      pbar.set_description(description)

    return learning_acc

In [ ]:
epochs = 20
best_acc = 0

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(trainloader,resnet50,criterion,optimizer_ft)
  acc = test(testloader,resnet50,criterion)
  if acc > best_acc:
    best_acc = acc
    torch.save(resnet50.state_dict(),f'/content/drive/MyDrive/ColabNotebooks/maskmodel_{acc}.pt') #(모델의 현재 파라미터저장,저장경로)